In [1]:
from pathlib import Path
import pandas as pd
from matplotlib import pyplot as plt
import json

In [3]:
import time
start_time = time.time()

In [4]:
TRAIN_PATH = Path("/home/ubuntu/data-1/nybg2020/train/")

In [5]:
! ls $TRAIN_PATH

images	metadata.json


In [6]:
with open(TRAIN_PATH/"metadata.json", encoding="utf8", errors='ignore') as json_file:
    data = json.load(json_file)

In [7]:
for i in data.keys():
    print(f'{i}: {len(data[i])}')

annotations: 1030747
categories: 32094
images: 1030747
info: 6
licenses: 1
regions: 4


In [8]:
df_ann =  pd.DataFrame.from_dict(data["annotations"])

In [9]:
df_imgs =  pd.DataFrame.from_dict(data["images"])

In [10]:
df_cat =  pd.DataFrame.from_dict(data["categories"])

In [11]:
df_cat.head()

,family,genus,id,name
0,Orchidaceae,Aa,0,Aa mathewsii (Rchb.f.) Schltr.
1,Orchidaceae,Aa,1,Aa paleacea (Kunth) Rchb.f.
2,Fabaceae,Abarema,2,Abarema abbottii (Rose & Leonard) Barneby & J....
3,Fabaceae,Abarema,3,Abarema acreana (J.F.Macbr.) L.Rico
4,Fabaceae,Abarema,4,Abarema adenophora (Ducke) Barneby & J.W.Grimes


In [12]:
df_merged = pd.merge(df_ann,df_imgs,on="id")
df_merged.to_csv("train_data.csv")
# df_merged = pd.merge(df_ann,df_imgs,left_on="image_id", right_on="id")

In [13]:
df_merged = pd.merge(df_merged,df_cat, left_on="category_id", right_on="id")

In [14]:
df_merged = df_merged.drop(columns=["id_x", "id_y"]) #get rid of repeated data

In [15]:
# df_merged = df_merged.rename(columns={"category_id":"label", "file_name":"name", "name":"class_name"})

In [16]:
 df_merged.head()

,category_id,image_id,region_id,file_name,height,license,width,family,genus,name
0,15672,354106,1,images/156/72/354106.jpg,1000,1,661,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg
1,15672,545181,1,images/156/72/545181.jpg,1000,1,661,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg
2,15672,449419,1,images/156/72/449419.jpg,1000,1,662,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg
3,15672,200223,1,images/156/72/200223.jpg,1000,1,661,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg
4,15672,5327,1,images/156/72/5327.jpg,1000,1,661,Lecythidaceae,Lecythis,Lecythis retusa Spruce ex O.Berg


In [17]:
family_ids_map = {n:i for i,n in enumerate(set(df_merged["family"]))}
family_ids = [family_ids_map[i] for i in list(df_merged["family"]) ]

In [18]:
len(family_ids), len(df_merged)

(1030747, 1030747)

In [19]:
genus_ids_map = {n:i for i,n in enumerate(set(df_merged["genus"]))}
genus_ids = [genus_ids_map[i] for i in list(df_merged["genus"]) ]

In [20]:
len(family_ids), len(df_merged)

(1030747, 1030747)

In [21]:
df_merged["family_id"] = family_ids
df_merged["genus_id"] = genus_ids

In [22]:
df_merged.tail()

,category_id,image_id,region_id,file_name,height,license,width,family,genus,name,family_id,genus_id
1030742,31957,729305,3,images/319/57/729305.jpg,1000,1,667,Rutaceae,Zanthoxylum,Zanthoxylum dumosum A.Rich.,88,2672
1030743,3834,574147,0,images/038/34/574147.jpg,1000,1,745,Asteraceae,Brickellia,Brickellia cuspidata A.Gray,175,1341
1030744,3834,853499,0,images/038/34/853499.jpg,1000,1,706,Asteraceae,Brickellia,Brickellia cuspidata A.Gray,175,1341
1030745,17312,964931,1,images/173/12/964931.jpg,1000,1,667,Marantaceae,Maranta,Maranta parvifolia Petersen,10,1391
1030746,17312,637634,1,images/173/12/637634.jpg,1000,1,667,Marantaceae,Maranta,Maranta parvifolia Petersen,10,1391


In [23]:
df_merged.to_csv(TRAIN_PATH/"train.csv")

In [ ]:
df_merged["genus"].value_counts()

In [ ]:
df_merged["family"].value_counts()

Look at one sample from each region
Might be an options to train region classifier that may help narrow down selecting which class it is
Triplet loss, maybe use look for whale tale for some ideas

In [ ]:
# sample = df_merged.sample(n=2000).drop_duplicates(['region_id'])
# f_names = list(sample["file_name"])

In [ ]:
# from PIL import Image

In [ ]:
# im0 = Image.open(TRAIN_PATH/f_names[0])
# im1 = Image.open(TRAIN_PATH/f_names[1])
# im2 = Image.open(TRAIN_PATH/f_names[2])
# im3 = Image.open(TRAIN_PATH/f_names[3])

In [ ]:
# f, axarr = plt.subplots(2,2)
# axarr[0,0].imshow(im0)
# axarr[0,1].imshow(im1)
# axarr[1,0].imshow(im2)
# axarr[1,1].imshow(im3)


In [ ]:
#plt.imshow(im1)

Game plan
- pretrain on identifying region ?
- look at techniquqes used in the whawle identification stuff
- possibly use simese network

In [3]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [1]:
from fastai.vision import *

In [ ]:
# df_train = df_merged[["file_name", "family_id"]]

In [4]:
df_train.head()

,file_name,category_id
0,images/156/72/354106.jpg,15672
1,images/156/72/545181.jpg,15672
2,images/156/72/449419.jpg,15672
3,images/156/72/200223.jpg,15672
4,images/156/72/5327.jpg,15672


In [5]:
df_train = df_train.rename(columns={"category_id": "label", "file_name":" name"})

In [6]:
new_df = df_train.groupby('label').apply(lambda x: x.sample(n=2, replace=True)).reset_index(drop = True)

In [ ]:
len(new_df)

In [8]:
data = (ImageList.from_df(df_train,TRAIN_PATH)
        .split_by_rand_pct()
        .label_from_df()
        .transform(size=256)
        .databunch(bs=128))

/home/shaun/personal/.pytorch-env/lib/python3.6/site-packages/fastai/data_block.py:534: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
9174, 16909, 22856, 7822, 8928...
  if getattr(ds, 'warn', False): warn(ds.warn)


In [ ]:
data.show_batch()

In [9]:
learn = cnn_learner(data, models.resnet18, metrics=accuracy)

In [10]:
learn.summary

<bound method model_summary of Learner(data=ImageDataBunch;

Train: LabelList (824598 items)
x: ImageList
Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256)
y: CategoryList
15672,15672,15672,15672,15672
Path: /home/shaun/personal/kaggle-data/nybg2020/train;

Valid: LabelList (205773 items)
x: ImageList
Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256)
y: CategoryList
29840,2761,10733,2220,2428
Path: /home/shaun/personal/kaggle-data/nybg2020/train;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddin

In [ ]:
learn.fit(2)

In [ ]:
learn.save("model-1")

In [ ]:
time.time() - start_time